In [1]:
# Please Open this File in Jupyter Notebook.

# Import necessary libraries
import re
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from IPython.core.display import display,HTML

In [2]:
urls = ['https://www.icliniq.com/search/online-doctors-directory',
        'https://www.icliniq.com/search/online-doctors-directory?page=2',
        'https://www.icliniq.com/search/online-doctors-directory?page=3']

# Global variables
Name = []
Photo = []
Degree = []
Specialization = []
Experience = []
Clinic_Name = []
Language = []
Rating = []
Query_Fee = []
Phone_Fee = []
Address = []
Profile_Link = []

class Doctors:
    # Initialize Global Variables
    global Name, Photo, Degree, Specialization, Experience, Language, Rating, Query_Free, Phone_Fee
    global Address_Profile_Link
    
    @staticmethod
    def scrap_details():
        # target urls
        for url in urls:
            page = requests.get(url)

            soup = BeautifulSoup(page.content, "html.parser")
            results = soup.find(class_ ="ic-doctor-list")
        
            # Doctor Names
            names = results.find_all("h3", class_="case-study")
            for name in names:
                Name.append(name.find('a').text)
        
            # Doctors Rating
            rating = results.find_all("div", class_="overall-rating")
            for rate in rating:
                Rating.append(re.findall(r'[-+]?\d*\.\d+|\d+', rate.text)[0])
        
            # Doctors Experience
            experience = results.find_all('p')
            for i in experience:
                if i.find('p', class_="m-0") == None:
                    continue
                else:
                    Experience.append(re.findall(r'[0-9]{1,2}', i.find('span', class_="font-weight-bold").text)[0] + "+ yrs")
    
            #Doctors specialization
            specializations = results.find_all('div', class_="mb-2")
            for i in specializations:
                if i.find('a').text == "Next Page":
                    continue
                else:
                    Specialization.append(i.find('a').text)
        
            # languages 
            languages = results.find_all('div', class_="grid-display")
            for i in languages:
                Language.append(i.find('p', class_="m-0").find_next('p').find_next('p').
                                 find_next('p').find('span', class_="font-weight-bold").text)
    
            links = results.find_all('div', class_="media-body")
            for link in links:
                if link.find('a', attrs={'href': re.compile("^/doctor/")}) == None:
                    continue
                else:
                    Profile_Link.append("https://www.icliniq.com"+link.find('a', attrs={'href': re.compile("^/doctor/")})
                                     .get('href'))
                    
            # Clinic Name
            for link in Profile_Link:
                page = requests.get(link)
                soup = BeautifulSoup(page.content, "html.parser")
                results = soup.find(class_="container-fluid pt-2")
                
                clinic = results.find_all('div', class_="addressDiv p-3")
                for i in clinic:
                    Clinic_Name.append(i.find_next('p').text)
            
            # degrees 
            for link in Profile_Link:
                page = requests.get(link)
                soup = BeautifulSoup(page.content, "html.parser")
                results = soup.find(class_="container-fluid pt-2")

                degree = results.find_all('div', class_="media-body align-self-center pl-1")
                for i in degree:
                    Degree.append((i.find('span', class_="small").text).strip())
            
            # address 
            for link in Profile_Link:
                page = requests.get(link)
                soup = BeautifulSoup(page.content, "html.parser")
                results = soup.find(class_="container-fluid pt-2")

                addresses = results.find_all('div', class_="loaction mt-1")
                for i in addresses:
                    Address.append((i.find('p').find_next('p').text).strip())
            
            # fees
            for link in Profile_Link:
                page = requests.get(link)
                soup = BeautifulSoup(page.content, "html.parser")
                results = soup.find(class_="container-fluid pt-2")

                queries = results.find_all('div', class_="mb-3 font-weight-bold")
                for i in queries:
                    Query_Fee.append(re.findall(r"[Rs]+.\d+" ,i.find_next('p').text)[0])
                    Phone_Fee.append(re.findall(r"[Rs]+.\d+", i.find_next('p').find_next('p').text)[0])
    
            #photos
            for link in Profile_Link:
                page = requests.get(link)
                soup = BeautifulSoup(page.content, "html.parser")
                results = soup.find(class_="container-fluid pt-2")

                photos = results.find_all('div', class_="d-flex justify-content-center")

                for i in photos:
                    Photo.append(i.find('img').get('src'))
                    
        return "All Details Has Been Extracted."
        
Doctors().scrap_details()

'All Details Has Been Extracted.'

In [29]:
# First 70 records
i =70
records = {'Name' : Name[:i], 'Photo' : Photo[:i], 'Degree' : Degree[:i], 'Specialization': Specialization[:i], 
           'Experience' : Experience[:i], 'Clinic Name' : Clinic_Name[:i], 'Language' : Language[:i], 'Rating' : Rating[:i],
           'Address' : Address[:i],'Query Fee' : Query_Fee[:i], 'Phone Fee' : Phone_Fee[:i], 
           'Link To Profile' : Profile_Link[:i]}

data = pd.DataFrame(records)
data.index = np.arange(1,71,1)

# displaying images in dataframe
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

pd.set_option('display.max_colwidth', None)
image_cols = ['Photo']
format_dict = {}
for image_col in image_cols:
    format_dict[image_col] = path_to_image_html

display(HTML(data.to_html(escape=False ,formatters=format_dict)))

,Name,Photo,Degree,Specialization,Experience,Clinic Name,Language,Rating,Address,Query Fee,Phone Fee,Link To Profile
1,Dr. Reetika Joshi,,"MBBS., DGO., NBE OBSTETRICS AND GYNAECOLOGY",Obstetrics And Gynaecology,20+ yrs,Dwarka,"English, Hindi, Punjabi",5,"Opp Maxfort School, Delhi, Delhi, Zip: 110075, IN",Rs.199,Rs.399,https://www.icliniq.com/doctor/dr-reetika-joshi
2,Dr. J N Naidu,,"MBBS., MD BIOCHEMISTRY",Family Physician,15+ yrs,BAPTIST CHRISTIANITY HOSPITAL,English,5,"Near Mission Chiriali Opp Tezpur Police Station, No.1 Thakurbari T.e., Assam, Zip: 784001, IN",Rs.199,Rs.399,https://www.icliniq.com/doctor/dr-j-n-naidu
3,Dr. Mohamad Ali Rida,,"LUPUS AND PSORIATIC ARTHRITIS FELLOWSHIP., RHEUMATOLOGY FELLOWSHIP., INTERNAL MEDICINE",Rheumatology,7+ yrs,BAPTIST CHRISTIAN HOSPITAL,English,4.5,"Mission Chiriali Keketibari Tezpur Assam, No.1 Thakurbari T.e., Assam, Zip: 784001, IN",Rs.969,Rs.1929,https://www.icliniq.com/doctor/dr-mohamad-ali-rida
4,Dr. Mohammad Saquib Alam,,MBBS,General Practitioner,2+ yrs,sanjeevani pharmaco,English,4.2,"Hathi Pilkhana , Assam, Zip: 784001, IN",Rs.200,Rs.399,https://www.icliniq.com/doctor/dr-mohammad-saquib-alam
5,Dr. Seikhoo Bishnoi,,"MBBS., MD PSYCHIATRY",Psychiatry,25+ yrs,Dhatri clinic,Hindi,4.53,"S V Nagar Main Road, Tirupati West, Andhra Pradesh, Zip: 517502, IN",Rs.399,Rs.699,https://www.icliniq.com/doctor/dr-seikhoo-bishnoi
6,Dr. Sandeep Shrestha,,MS OTORHINOLARYNGOLOGY,Otolaryngology (E.N.T),3+ yrs,Clemenceau medical center,English,5,"Clemenceau, Beirut, Lebanon, Zip: 00000, LB",Rs.320,Rs.640,https://www.icliniq.com/doctor/dr-sandeep-shrestha
7,Dr. Snehal Laul,,"DDV (DERMATOLOGY & VENEROLOGY)., M.B.B.S.",Dermatology,4+ yrs,Total Care,English,4.2,"Amu, Hathras, Uttar Pradesh, Zip: 204101, IN",Rs.199,Rs.399,https://www.icliniq.com/doctor/dr-snehal-laul
8,Dr. Sameer Kumar,,"MBBS., MS (OBS/GYNAE)., FMAS",Family Physician,16+ yrs,Primary Contact,"English, Hindi",4.59,"Maharaja Agrasen Medical College, Hisar, Haryana, Zip: 125001, IN",Rs.700,Rs.1000,https://www.icliniq.com/doctor/dr-sameer-kumar
9,Dr. Gokul Kruba Shanker,,"MBBS., MS., DNB(SURG GASTRO)",Surgical Gastroenterology,26+ yrs,NMCTH,"English, Tamil",3.8,"Attarkhel, Kathmandu, Bagmati, Zip: 44600, NP",Rs.1150,Rs.499,https://www.icliniq.com/doctor/drgokul-kruba-shanker-r
10,Sumiya Sulthana (Nutritionist),,MSC(NUTRITIONIST),Nutritionist,8+ yrs,Snehal's skin Clinic,English,4.3,"Dermatology And Venerology , Nashik, Maharashtra, Zip: 422003, IN",Rs.199,Rs.399,https://www.icliniq.com/doctor/dr-sumiya-sul1thana


In [30]:
# Convert to excel
data.to_excel('output.xlsx', index=False)